In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Ignore Harmless Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data Augmentation for training dataset

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(), # Default probability of 0.5
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], # Mean and Standard Deviation for each colour channels. Helps in faster convergence
                        [0.229, 0.224, 0.225])
])

# Data Augmentation for the testing dataset
# We dont need to perform random rotations or flips in the test dataset

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

root = 'C:\\Users\\shant\\CNN_Architectures_ImageNet_Pytorch\\Dataset\\CATS_DOGS\\'

# Location of the training data
train_data = datasets.ImageFolder(os.path.join(root,'train'),transform = train_transform)

# Location of the testing data
test_data = datasets.ImageFolder(os.path.join(root,'test'),transform = test_transform)

# Set a seed
torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10)

In [3]:
class VGGNet(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1)
        self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1)
        self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1)
        self.conv6 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1)
        self.conv7 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1)
        self.conv8 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1)
        self.conv9 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
        self.conv10 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
        self.conv11 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
        self.conv12 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
        self.conv13 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(7*7*512, 4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,2)
        
    def forward(self, X):
        
        X = F.relu(self.conv1(X))
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv5(X))
        X = F.relu(self.conv6(X))
        X = F.relu(self.conv7(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv8(X))
        X = F.relu(self.conv9(X))
        X = F.relu(self.conv10(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv11(X))
        X = F.relu(self.conv12(X))
        X = F.relu(self.conv13(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 7*7*512)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        
        return F.log_softmax(X, dim=1)

In [12]:
# Set the Seed
torch.manual_seed(101)

# Create Object of the Class
VGGNetModel = VGGNet()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

VGGNetModel.to(device)

# Declare the criterion
criterion = nn.CrossEntropyLoss()

# Declare the Optimizer
optimizer = torch.optim.Adam(VGGNetModel.parameters(), lr = 0.001)

In [13]:
torch.cuda.memory_allocated()

2560997376

In [14]:
# Training the Model

# Training and Evaluation of the Model

import time
start_time = time.time()

epochs = 2

# Limits on the number of batches
max_trn_batch = 800 # Batch of 10 images--> 8000 Images
max_tst_batch = 300 # Batch of 10 images--> 3000 Images

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train,y_train) in enumerate(train_loader):
        
        X_train = X_train.cuda()
        y_train = y_train.cuda()
        
        b += 1
        
        # Predicted Output
        y_pred = VGGNetModel(X_train)
        
        # Predicted Loss
        loss = criterion(y_pred,y_train)
        
        # Tally number of correct predictions
        predicted = torch.max(y_pred.data,1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update the Parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%200 == 0:
            print(f'Epoch {i} LOSS {loss.item()}')
    
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    #TEST SET
    
    with torch.no_grad():
        
        for b , (X_test,y_test) in enumerate(test_loader):
            
            X_test = X_test.cuda()
            y_test = y_test.cuda()
            
            y_val = VGGNetModel(X_test)
            
            predicted = torch.max(y_pred.data, 1)[1] # Give max value along each row and then present as a column vector
            batch_corr = (predicted == y_test).sum()
            tst_corr += batch_corr
            
    loss = criterion(y_val,y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
    
total_time = time.time() - start_time
print(f'Total Time: {total_time/60} minutes')

RuntimeError: CUDA out of memory. Tried to allocate 124.00 MiB (GPU 0; 4.00 GiB total capacity; 2.51 GiB already allocated; 97.90 MiB free; 2.56 GiB reserved in total by PyTorch)